In [115]:
#!pip install alembic

import pandas as pd
import numpy as np

#sqlalchemy 
import sqlalchemy
from sqlalchemy import create_engine, inspect, func, table
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String, Float, Text, DateTime
from sqlalchemy.ext.declarative import declarative_base


In [116]:

# Create the connection engine
engine = create_engine("sqlite:///data/sf_bike_share.sqlite")


#Base.metadata.create_all(engine)

In [117]:
#declaring auto_base
Base = automap_base()

#reflecting tables in the hawaii.db
Base.prepare(engine, reflect=True)

#gathering tables from DB
Base.classes.keys()

['station', 'status', 'trip']

In [118]:
#setting tables to variables
Station = Base.classes.station
Status = Base.classes.status
Trip = Base.classes.trip

In [119]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
inspector.get_table_names()

['station', 'status', 'trip', 'weather']

In [41]:
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
name TEXT
lat NUMERIC
long NUMERIC
dock_count INTEGER
city TEXT
installation_date TEXT


In [42]:
columns = inspector.get_columns('status')
for column in columns:
    print(column["name"], column["type"])

station_id INTEGER
bikes_available INTEGER
docks_available INTEGER
time TEXT
id INTEGER


In [43]:
columns = inspector.get_columns('trip')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
duration INTEGER
start_date TEXT
start_station_name TEXT
start_station_id INTEGER
end_date TEXT
end_station_name TEXT
end_station_id INTEGER
bike_id INTEGER
subscription_type TEXT
zip_code INTEGER


In [120]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

In [45]:
#checking the data in table
station_list = session.query(Station).limit(2)
for x in station_list:
    print(x.id)
    print(x.name)
    print(x.lat)
    print(x.long)
    print(x.dock_count)
    print(x.city)
    print(x.installation_date)

2
San Jose Diridon Caltrain Station
37.3297320000
-121.9017820000
27
San Jose
8/6/2013
3
San Jose Civic Center
37.3306980000
-121.8889790000
15
San Jose
8/5/2013


/anaconda3/lib/python3.6/site-packages/sqlalchemy/sql/sqltypes.py:596: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [101]:
#checking the data commit to DB
trip_list = session.query(Trip).limit(10)
for x in trip_list:
    print(x.id)
    print(x.duration)
    print(x.start_date)
    print(x.start_station_name)
    print(x.start_station_id)

4069
174
8/29/2013 9:08
2nd at South Park
64
4073
1067
8/29/2013 9:24
South Van Ness at Market
66
4074
1131
8/29/2013 9:24
South Van Ness at Market
66
4075
1117
8/29/2013 9:24
South Van Ness at Market
66
4076
1118
8/29/2013 9:25
South Van Ness at Market
66
4078
3829
8/29/2013 9:31
Redwood City Caltrain Station
22
4079
995
8/29/2013 9:35
South Van Ness at Market
66
4080
764
8/29/2013 9:36
South Van Ness at Market
66
4081
218
8/29/2013 9:38
Mountain View City Hall
27
4084
287
8/29/2013 9:41
Mountain View City Hall
27


In [71]:
data = session.query(Trip.start_date,Trip.start_station_id,Trip.start_station_name,func.count(Trip.start_station_id)).group_by(Trip.start_date,Trip.start_station_id,Trip.start_station_name).all()

In [72]:
datadf = pd.DataFrame(data, columns=['Month','Station ID','Station Name','Rides'])
datadf.head()

,Month,Station ID,Station Name,Rides
0,1/1/2014 0:14,58,San Francisco City Hall,2
1,1/1/2014 0:17,60,Embarcadero at Sansome,1
2,1/1/2014 0:23,74,Steuart at Market,2
3,1/1/2014 0:24,74,Steuart at Market,2
4,1/1/2014 0:25,57,5th at Howard,2


In [74]:
pd.to_datetime(datadf['Month'])

0        2014-01-01 00:14:00
1        2014-01-01 00:17:00
2        2014-01-01 00:23:00
3        2014-01-01 00:24:00
4        2014-01-01 00:25:00
5        2014-01-01 00:25:00
6        2014-01-01 00:28:00
7        2014-01-01 00:32:00
8        2014-01-01 00:33:00
9        2014-01-01 00:34:00
10       2014-01-01 00:36:00
11       2014-01-01 00:37:00
12       2014-01-01 00:39:00
13       2014-01-01 00:58:00
14       2014-01-01 10:07:00
15       2014-01-01 10:09:00
16       2014-01-01 10:11:00
17       2014-01-01 10:12:00
18       2014-01-01 10:14:00
19       2014-01-01 10:25:00
20       2014-01-01 10:26:00
21       2014-01-01 10:32:00
22       2014-01-01 10:32:00
23       2014-01-01 10:33:00
24       2014-01-01 10:34:00
25       2014-01-01 10:37:00
26       2014-01-01 10:38:00
27       2014-01-01 10:38:00
28       2014-01-01 10:39:00
29       2014-01-01 10:41:00
                 ...        
615545   2014-09-09 09:39:00
615546   2014-09-09 09:40:00
615547   2014-09-09 09:40:00
615548   2014-

In [78]:

datadf["Month"] = pd.to_datetime(datadf["Month"])

In [80]:
datadf['Month_up'] = datadf['Month'].dt.date

In [84]:
datadf = datadf[['Month_up','Month','Station ID','Station Name','Rides']]
datadf.head()

,Month_up,Month,Station ID,Station Name,Rides
0,2014-01-01,2014-01-01 00:14:00,58,San Francisco City Hall,2
1,2014-01-01,2014-01-01 00:17:00,60,Embarcadero at Sansome,1
2,2014-01-01,2014-01-01 00:23:00,74,Steuart at Market,2
3,2014-01-01,2014-01-01 00:24:00,74,Steuart at Market,2
4,2014-01-01,2014-01-01 00:25:00,57,5th at Howard,2


In [113]:
datadfGB = datadf.groupby(['Month_up','Station ID','Station Name'])
datagroupmonthstartdf = datadfGB.sum()

In [114]:
datagroupmonthstartdf.to_csv('output/month_rides_by_start_station.csv')

In [121]:
#gathering end station data from DB table
data = session.query(Trip.end_date,Trip.end_station_id,Trip.end_station_name,func.count(Trip.end_station_id)).group_by(Trip.end_date,Trip.end_station_id,Trip.end_station_name).all()

In [122]:
#loading query result into DF
dataendstationdf = pd.DataFrame(data, columns=['Month','Station ID','Station Name','Rides'])
dataendstationdf.head()

,Month,Station ID,Station Name,Rides
0,1/1/2014 0:21,65,Townsend at 7th,2
1,1/1/2014 0:35,68,Yerba Buena Center of the Arts (3rd @ Howard),2
2,1/1/2014 0:37,46,Washington at Kearney,3
3,1/1/2014 0:42,56,Beale at Market,1
4,1/1/2014 0:47,49,Spear at Folsom,1


In [123]:
#creating a new date column without timestamp
dataendstationdf["Month"] = pd.to_datetime(dataendstationdf["Month"])
dataendstationdf['Month_up'] = dataendstationdf['Month'].dt.date
#renaming columns
dataendstationdf = dataendstationdf[['Month_up','Month','Station ID','Station Name','Rides']]
dataendstationdf.head()

,Month_up,Month,Station ID,Station Name,Rides
0,2014-01-01,2014-01-01 00:21:00,65,Townsend at 7th,2
1,2014-01-01,2014-01-01 00:35:00,68,Yerba Buena Center of the Arts (3rd @ Howard),2
2,2014-01-01,2014-01-01 00:37:00,46,Washington at Kearney,3
3,2014-01-01,2014-01-01 00:42:00,56,Beale at Market,1
4,2014-01-01,2014-01-01 00:47:00,49,Spear at Folsom,1


In [124]:
#grouping columns to month, station ID, and station name
dataendstationdfGB = dataendstationdf.groupby(['Month_up','Station ID','Station Name'])
datagroupmonthenddf = dataendstationdfGB.sum()
datagroupmonthenddf.head()

Rides
Month_up   Station ID Station Name                            
2013-08-29 2          San Jose Diridon Caltrain Station      6
           3          San Jose Civic Center                  5
           4          Santa Clara at Almaden                 3
           5          Adobe on Almaden                       2
           6          San Pedro Square                       6

In [125]:
#exporting
datagroupmonthenddf.to_csv('output/month_rides_by_end_station.csv')